# Desafio +A Educação - Engenheiro de Inteligência Artificial

Este notebook contempla as etapas e soluções requeridas no desafio pontuadas no README.md na pasta raiz e estruturadas de acordo

*Autora*: Leticia Campos Valente

## Etapa 1: Indexação dos dados

**Enunciado**:\
"Defina a ferramenta de indexação que considere adequada para o projeto. Recomenda-se escolher uma ferramenta que possa suportar a geração dinâmica de conteúdos adaptativos no prompt de IA generativa. Indexe os diferentes tipos de dados para permitir uma busca eficiente e relevante."

**Conjunto de dados**: \
Textos: Uma coleção de textos extraídos de módulos de aprendizagem.
PDFs: Livros e manuais introdutórios sobre o conteúdo abordado.
Vídeos: Pequenos vídeos de dicas de professor, que explicam o conteúdo abordado. Os vídeos estão em formatos .mp4


Dadas as condições, opta-se pela ferramenta ElasticSearch pela eficiencia no armazenamento de documentação e integração em Langchain

 <u> Notas </u>

1. Existem nesse código mais de uma estratégia para realizar, por exemplo, extração das keywords. Isso se dá porque o mesmo método que performa bem para o texto de exemplo não performou tão bem para os textos em pdf, etc. Além disso, inclui-se mais abordagens propositalmente para demonstrar os conhecimentos em cada estratégia
2.

In [1]:
from elasticsearch import Elasticsearch

# Instanciando o Elasticsearch
es = Elasticsearch("http://localhost:9200")

### 1.1 Pré-processamento de Texto

É preciso primeiramente extrair as palavras chaves do texto para indexação conforme conteúdo e extração de frases relevantes. 

**Estratégia de extração de palavras-chave**: Term Frequency-Inverse Document Frequency (TF-IDF) \
**Estratégia de extração de frases mais relevantes**: Sentence ranking via spacy 

Escolheu-se utilizar abordagens de NLP simples ao invés do uso de um modelo pré-treinado BERT, por exemplo, por mera questão de simplicidade e eficiência para este documento de exemplo, mas em textos mais complexos, a autora consideraria a segunda opção.

As funções se encontram no módulo source/text_utils.py

In [2]:
import nltk
from text_utils import latin_text_preprocessing, tfidf_keyword_extraction, text_most_relevant_prhases_extraction, extract_programming_language

In [3]:
%%capture
!python -m spacy download pt_core_news_sm

In [4]:
%%capture
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mototaxista\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mototaxista\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##### Execução

In [5]:
with open('../resources/Apresentação.txt', 'r') as text_file:
    text_data = text_file.read()

In [6]:
text_doc = {
    'content':str(latin_text_preprocessing(text_data)),
    'keywords':tfidf_keyword_extraction(text_data),
    'relevant_phrases':text_most_relevant_prhases_extraction(text_data),
    'programming_languages':extract_programming_language(text_data)
}

In [7]:
text_doc

{'content': 'Sistemas desenvolvidos para a plataforma web utilizam a linguagem HTML para a exibição de conteúdo, tanto para páginas estáticas como dinâmicas. Com HTML5 é possível criar páginas web com diversos recursos para a apresentação de dados por meio de novas marcações que permitem o uso de semântica e acessibilidade, facilitando a pesquisa por motores de busca automática e dispositivos próprios para deficientes visuais e auditivos.\n\nNesta Unidade de Aprendizagem, você vai estudar a estrutura de páginas web, a formatação de texto em documentos hipertexto e a apresentação de links, listas e tabelas em HTML5.\n\nBons estudos.\n\nAo final desta Unidade de Aprendizagem, você deve apresentar os seguintes aprendizados:\nDefinir a estrutura de uma página web com HTML5.\nAplicar formatação de texto em uma página web com HTML5.\nDesenvolver listas e tabelas em uma página web com HTML5.',
 'keywords': ['página',
  'web',
  'html5',
  'aprendizagem',
  'apresentação',
  'desenvolver',
  '

In [8]:
# es.indices.create(index='textos', body={
#     'mappings': {
#         'properties': {
#             'conteudo': {'type': 'text'},
#             'keywords': {'type': 'keyword'},
#             'relevant_phrases': {'type': 'keyword'},
#         }
#     }
#     }, ignore=400)

# es.index(index='textos', body=text_doc)

### 1.2 Pré-processamento de PDF

**Estratégia de extração de palavras-chave**: YAKE (Yet another keyword extractor) \
**Estratégia de extração de autor do texto**: NER ( Named Entity Recognition)


As funções se encontram no módulo source/text_utils.py

In [9]:
from text_utils import yake_keyword_extraction, ner_author_extractor
from PyPDF2 import PdfReader

##### Execução

In [10]:
with open('../resources/Capítulo do Livro.pdf', 'rb') as pdf_file:
    # Criação do objeto PdfReader
    pdf_reader = PdfReader(pdf_file)

    extracted_text = ""

    # Extração do texto do pdf por páginas
    for page_num in range(len(pdf_reader.pages)):
        page = pdf_reader.pages[page_num]
        extracted_text += page.extract_text()

unknown widths : 
[0, IndirectObject(176, 0, 1701729483344)]


In [11]:
pdf_doc = {
    'author':ner_author_extractor(extracted_text),
    'content':extracted_text,
    'keywords':yake_keyword_extraction(extracted_text),
    'programming_languages':extract_programming_language(extracted_text)
}

In [12]:
pdf_doc

{'author': 'Maurício de Oliveira Saraiva',
 'content': 'DESENVOLVIMENTO \nDE SISTEMAS PHP \nMaurício de Oliveira Saraiva Criação de páginas web \ncom HTML5\nObjetivos de aprendizagem\nAo final deste texto, você deve apresentar os seguintes aprendizados:\n \x84Definir a estrutura de uma página web com HTML5.\n \x84Aplicar a formatação de texto em uma página web com HTML5.\n \x84Desenvolver listas e tabelas em uma página web com HTML5.\nIntrodução\nSistemas desenvolvidos para a plataforma web, tanto para páginas es -\ntáticas como dinâmicas, utilizam a linguagem HTML para a exibição \ndo conteúdo. Com HTML5, é possível criar páginas web com diversos \nrecursos para a apresentação de dados, por meio de novas marcações \nque permitem o uso de semântica e acessibilidade. Isso facilita a pesquisa \npor motores de busca automática, dispositivos próprios para deficientes \nvisuais e auditivos, entre outros.\nNeste capítulo, você vai estudar a estrutura de páginas web, a for -\nmatação de texto

### 1.3 Pré-processamento de vídeos

#### 1.3.1 Extração do áudio e transcrição de áudio para texto

In [13]:
from audio_video_utils import extract_audio, extract_video_metadata, transcribe_audio

##### Execução

In [14]:
video_file_path = "../resources/Dica do professor.mp4"
audio_file_path = "../resources/Audio_Dica_do_Professor.wav"

In [15]:
# # Extração dos metadados do vídeo
# video_doc = extract_video_metadata(video_file_path)

# # Extração do áudio em .wav
# extract_audio(video_file_path, audio_file_path)

# # Extração da transcrição do vídeo
# transcript = transcribe_audio(audio_file_path)

# video_doc['content'] = transcript
# video_doc['keywords']=tfidf_keyword_extraction(transcript),
# video_doc['programming_languages']=extract_programming_language(transcript)

In [16]:
#video_doc

### 1.4 Pré-processamento de imagens

In [17]:
import torch
from image_utils import get_image_description, get_exif_data, extract_text_from_image

ImportError: cannot import name 'extract_text_from_image' from 'image_utils' (c:\Users\Mototaxista\Desktop\Git gen ai\challenge-artificial-intelligence\source\image_utils.py)

In [ ]:



def process_image(image_path):
    # Descrição do conteúdo visual
    description = get_image_description(image_path)
    print(f'Descrição: {description}')
    
    # Texto extraído da imagem
    text = extract_text_from_image(image_path)
    print(f'Texto: {text}')
    
    # Metadados EXIF
    exif_data = get_exif_data(image_path)

    print(f'Metadados EXIF: {exif_data}')

# Exemplo de uso
image_path = '../resources/Infografico-1.jpg'
process_image(image_path)

Descrição: captura de ecrã de um ecrã de computador que apresenta uma lista de diferentes idiomas


NameError: name 'Image' is not defined

In [ ]:
import pytesseract
from PIL import Image

def extract_text_from_image(image_path):
    # Open the image
    img = Image.open(image_path)
    
    # Perform OCR using Tesseract
    text = pytesseract.image_to_string(img, lang='por')  # Language code for Portuguese
    
    return text

# Path to your image
image_path = '../resources/Infografico-1.jpg'

# Extract text from image
extracted_text = extract_text_from_image(image_path)

print("Extracted Text:", extracted_text)


TesseractNotFoundError: tesseract is not installed or it's not in your PATH. See README file for more information.